In [1]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()


In [2]:

spotify = spark.read.format('delta').load('/home/mlops/project/DeltaLake/gold_data/spotify_cleansed_table')
spotify.to_pandas_on_spark().head()

,href,uri,popularity,followers,id,name,genres,type,uid,create_time,update_time,read_time,url,image_url
0,https://api.spotify.com/v1/artists/711MCceyCBc...,spotify:artist:711MCceyCBcFnzjGY4Q7Un,83,21527801,711MCceyCBcFnzjGY4Q7Un,AC/DC,"[australian rock, hard rock, rock]",artist,uagVX8DeaePP88Qz5Vj2EtXstK03,2021-12-11 12:21:02.186210,2021-12-11 12:21:02.186210,2022-01-09 16:38:48.566406,https://open.spotify.com/artist/711MCceyCBcFnz...,https://i.scdn.co/image/ab6761610000e5ebc4c775...
1,https://api.spotify.com/v1/artists/3JsMj0DEzyW...,spotify:artist:3JsMj0DEzyWc0VDlHuy9Bx,71,2450429,3JsMj0DEzyWc0VDlHuy9Bx,Supertramp,"[album rock, art rock, classic rock, glam rock...",artist,uagVX8DeaePP88Qz5Vj2EtXstK03,2021-12-11 12:21:02.186210,2021-12-11 12:21:02.186210,2022-01-09 16:38:48.566406,https://open.spotify.com/artist/3JsMj0DEzyWc0V...,https://i.scdn.co/image/eebed895cad509b17c7c6f...
2,https://api.spotify.com/v1/artists/11YTrgsi6Vj...,spotify:artist:11YTrgsi6VjRCkExQBY13k,50,115537,11YTrgsi6VjRCkExQBY13k,Wise Guys,"[german pop, german rock, kabarett, liedermacher]",artist,uagVX8DeaePP88Qz5Vj2EtXstK03,2021-12-11 12:21:02.186210,2021-12-11 12:21:02.186210,2022-01-09 16:38:48.566406,https://open.spotify.com/artist/11YTrgsi6VjRCk...,https://i.scdn.co/image/4f1b2cabff7abd06fb17ed...
3,https://api.spotify.com/v1/artists/492hDmhPyuI...,spotify:artist:492hDmhPyuIjP3MgTcIqgm,55,301844,492hDmhPyuIjP3MgTcIqgm,Caro Emerald,"[dutch pop, electro swing, nu jazz]",artist,uagVX8DeaePP88Qz5Vj2EtXstK03,2021-12-11 12:21:02.186210,2021-12-11 12:21:02.186210,2022-01-09 16:38:48.566406,https://open.spotify.com/artist/492hDmhPyuIjP3...,https://i.scdn.co/image/ab6761610000e5eb379e88...
4,https://api.spotify.com/v1/artists/06bDwgCHeMA...,spotify:artist:06bDwgCHeMAwhgI8il4Y5k,69,708011,06bDwgCHeMAwhgI8il4Y5k,Set It Off,"[electropowerpop, modern rock, otacore, pixie,...",artist,uagVX8DeaePP88Qz5Vj2EtXstK03,2021-12-11 12:21:02.186210,2021-12-11 12:21:02.186210,2022-01-09 16:38:48.566406,https://open.spotify.com/artist/06bDwgCHeMAwhg...,https://i.scdn.co/image/ab6761610000e5eba368e7...


In [3]:
genre_map = spark.read.format('delta').load('/home/mlops/project/DeltaLake/bronze_data/genre_map_table')
genre_map.show()

+--------------+--------------------+--------------------+
|        genres|       main_genre_18|        main_genre_9|
+--------------+--------------------+--------------------+
|           pop|                 Pop|                 Pop|
|     dance pop|                 Pop|                 Pop|
|           rap|     Rap and Hip-Hop|     Rap and Hip-Hop|
|          rock|                Rock|Rock and Heavy Metal|
|         latin|Latin American music|Latin American music|
|       pop rap|                 Pop|                 Pop|
|       hip hop|     Rap and Hip-Hop|     Rap and Hip-Hop|
|   trap latino|Latin American music|Latin American music|
|          trap|     Rap and Hip-Hop|     Rap and Hip-Hop|
|   modern rock|                Rock|Rock and Heavy Metal|
|           edm|Dance Music, Tech...|  Electronical music|
| post-teen pop|                 Pop|                 Pop|
|     reggaeton|Latin American music|Latin American music|
|     pop dance|Dance Music, Tech...|  Electronical musi

In [4]:
from pyspark.sql.types import Row
spotify_aggregate = spotify

genre_list = genre_map.to_pandas_on_spark().to_dict('list')

def f(row):
    genre_set_9 = set()
    genre_set_18 = set()
    for x in row.genres:
        if x in genre_list['genres']:
            index = genre_list['genres'].index(x)
            genre_set_9.add(genre_list['main_genre_9'][index])
            genre_set_18.add(genre_list['main_genre_18'][index])
    row = row.asDict()
    row['genres_9'] = list(genre_set_9)
    row['genres_18'] = list(genre_set_18)
    return Row(**row)

spotify_aggregate = spotify_aggregate.rdd.map(f)
spotify_aggregate = spotify_aggregate.toDF()
spotify_aggregate.to_pandas_on_spark().head()

,href,uri,popularity,followers,id,name,genres,type,uid,create_time,update_time,read_time,url,image_url,genres_9,genres_18
0,https://api.spotify.com/v1/artists/711MCceyCBc...,spotify:artist:711MCceyCBcFnzjGY4Q7Un,83,21527801,711MCceyCBcFnzjGY4Q7Un,AC/DC,"[australian rock, hard rock, rock]",artist,uagVX8DeaePP88Qz5Vj2EtXstK03,2021-12-11 12:21:02.186210,2021-12-11 12:21:02.186210,2022-01-09 16:38:48.566406,https://open.spotify.com/artist/711MCceyCBcFnz...,https://i.scdn.co/image/ab6761610000e5ebc4c775...,[Rock and Heavy Metal],[Rock]
1,https://api.spotify.com/v1/artists/3JsMj0DEzyW...,spotify:artist:3JsMj0DEzyWc0VDlHuy9Bx,71,2450429,3JsMj0DEzyWc0VDlHuy9Bx,Supertramp,"[album rock, art rock, classic rock, glam rock...",artist,uagVX8DeaePP88Qz5Vj2EtXstK03,2021-12-11 12:21:02.186210,2021-12-11 12:21:02.186210,2022-01-09 16:38:48.566406,https://open.spotify.com/artist/3JsMj0DEzyWc0V...,https://i.scdn.co/image/eebed895cad509b17c7c6f...,[Rock and Heavy Metal],[Rock]
2,https://api.spotify.com/v1/artists/11YTrgsi6Vj...,spotify:artist:11YTrgsi6VjRCkExQBY13k,50,115537,11YTrgsi6VjRCkExQBY13k,Wise Guys,"[german pop, german rock, kabarett, liedermacher]",artist,uagVX8DeaePP88Qz5Vj2EtXstK03,2021-12-11 12:21:02.186210,2021-12-11 12:21:02.186210,2022-01-09 16:38:48.566406,https://open.spotify.com/artist/11YTrgsi6VjRCk...,https://i.scdn.co/image/4f1b2cabff7abd06fb17ed...,"[Alternative, Rock and Heavy Metal, Not_assign...","[Not_assignable, Utility, Rock, Pop]"
3,https://api.spotify.com/v1/artists/492hDmhPyuI...,spotify:artist:492hDmhPyuIjP3MgTcIqgm,55,301844,492hDmhPyuIjP3MgTcIqgm,Caro Emerald,"[dutch pop, electro swing, nu jazz]",artist,uagVX8DeaePP88Qz5Vj2EtXstK03,2021-12-11 12:21:02.186210,2021-12-11 12:21:02.186210,2022-01-09 16:38:48.566406,https://open.spotify.com/artist/492hDmhPyuIjP3...,https://i.scdn.co/image/ab6761610000e5eb379e88...,"[Rhythm music, Electronical music, Pop]","[Jazz, Pop, Dance Music, Techno and House]"
4,https://api.spotify.com/v1/artists/06bDwgCHeMA...,spotify:artist:06bDwgCHeMAwhgI8il4Y5k,69,708011,06bDwgCHeMAwhgI8il4Y5k,Set It Off,"[electropowerpop, modern rock, otacore, pixie,...",artist,uagVX8DeaePP88Qz5Vj2EtXstK03,2021-12-11 12:21:02.186210,2021-12-11 12:21:02.186210,2022-01-09 16:38:48.566406,https://open.spotify.com/artist/06bDwgCHeMAwhg...,https://i.scdn.co/image/ab6761610000e5eba368e7...,"[Alternative, Rock and Heavy Metal, Not_assign...","[Not_assignable, Utility, Rock, Pop]"


In [5]:
from pyspark.sql.types import Row
from pyspark.sql.functions import explode, collect_list, mean
from collections import Counter


spotify_unify = spotify_aggregate

spotify_unify_followers = spotify_unify.groupby('uid', 'create_time', 'update_time', 'read_time').agg(mean('followers').alias("followers"))
spotify_unify_popularity = spotify_unify.groupby('uid', 'create_time', 'update_time', 'read_time').agg(mean('popularity').alias("popularity"))

spotify_unify_9 = spotify_unify.select('*', explode(spotify_unify.genres_9).alias("genre"))
spotify_unify_9 = spotify_unify_9.groupby('uid', 'create_time', 'update_time', 'read_time').agg(collect_list('genre').alias("genres_9"))
spotify_unify_18 = spotify_unify.select('*', explode(spotify_unify.genres_18).alias("genre"))
spotify_unify_18 = spotify_unify_18.groupby('uid', 'create_time', 'update_time', 'read_time').agg(collect_list('genre').alias("genres_18"))
spotify_unify = spotify_unify.select('*', explode(spotify_unify.genres).alias("genre"))
spotify_unify = spotify_unify.groupby('uid', 'create_time', 'update_time', 'read_time').agg(collect_list('genre').alias("genres"))

spotify_unify = spotify_unify.join(spotify_unify_9, ['uid', 'create_time', 'update_time', 'read_time'])\
    .join(spotify_unify_18, ['uid', 'create_time', 'update_time', 'read_time'])\
    .join(spotify_unify_followers, ['uid', 'create_time', 'update_time', 'read_time'])\
    .join(spotify_unify_popularity, ['uid', 'create_time', 'update_time', 'read_time'])

def f(row):
    row = row.asDict()
    row['genres'] = dict(Counter(row['genres']))
    row['genres_9'] = dict(Counter(row['genres_9']))
    row['genres_18'] = dict(Counter(row['genres_18']))
    return Row(**row)

spotify_unify = spotify_unify.rdd.map(f)

spotify_unify = spotify_unify.toDF()

spotify_unify.to_pandas_on_spark()

,uid,create_time,update_time,read_time,genres,genres_9,genres_18,followers,popularity
0,fiFDr5jQg0aUSfSHw59g1rQFVSs1,2021-12-29 14:41:10.781769,2021-12-29 14:41:45.770151,2022-01-09 16:38:48.566406,"{'social media pop': 2, 'country': 1, 'rebel b...","{'Rhythm music': 4, 'Pop': 21, 'Alternative': ...","{'Rock': 16, 'Dance Music, Techno and House': ...",2.693735e+06,65.066667
1,Q3t2J73kpIcMqGBmB1ZEqLANtDz1,2021-12-06 18:23:20.788728,2022-01-08 11:26:15.073020,2022-01-09 16:38:48.566406,"{'miami hip hop': 3, 'nz pop': 1, 'piano rock'...","{'Rhythm music': 1, 'Pop': 30, 'Alternative': ...","{'Pop': 30, 'Utility': 1, 'Not_assignable': 5,...",1.167529e+07,77.608696
2,VQEI5Ea2AsP2tjLTlfBcmL0wn7t2,2021-12-06 15:01:05.156199,2022-01-09 15:39:58.557586,2022-01-09 16:38:48.566406,"{'social media pop': 4, 'candy pop': 1, 'nouve...","{'Rhythm music': 1, 'Pop': 25, 'Alternative': ...","{'Heavy Metal': 10, 'Pop': 25, 'Utility': 2, '...",7.156937e+06,70.812500
3,uagVX8DeaePP88Qz5Vj2EtXstK03,2021-12-11 12:21:02.186210,2021-12-11 12:21:02.186210,2022-01-09 16:38:48.566406,"{'finnish heavy metal': 1, 'rock-and-roll': 1,...","{'Rhythm music': 3, 'Pop': 10, 'Alternative': ...","{'Heavy Metal': 21, 'Pop': 9, 'Industrial and ...",4.858708e+06,69.702128
4,UrZpLI2DysWJIVCmLNwTW5JJJO62,2022-01-09 15:36:20.930481,2022-01-09 15:36:20.930481,2022-01-09 16:38:48.566406,"{'new rave': 1, 'filter house': 2, 'japanese c...","{'Rhythm music': 4, 'Pop': 11, 'Alternative': ...","{'Rock': 14, 'Dance Music, Techno and House': ...",2.468938e+06,63.487805
5,f6TABm8ygYT5P1NbtF7DQEk6bv82,2021-12-06 18:36:31.549954,2021-12-25 13:56:33.876422,2022-01-09 16:38:48.566406,"{'melodic hard rock': 1, 'italian pop': 1, 'in...","{'Pop': 1, 'Rock and Heavy Metal': 2}","{'Pop': 1, 'Rock': 2}",2.492494e+06,53.500000


In [6]:
# Extract top genre
from pyspark.sql.functions import filter

spotify_extracted = spotify_unify

def f(row):
    row = row.asDict()
    highest_number = 0
    highest_label = 'undefined'
    for genre  in row['genres_9'].items():
        if genre[1] > highest_number:
            highest_number = genre[1]
            highest_label = genre[0]
    row['genres_1'] = highest_label
    return Row(**row)

spotify_extracted = spotify_extracted.rdd.map(f)

spotify_extracted =spotify_extracted.toDF()

spotify_extracted.to_pandas_on_spark()

,uid,create_time,update_time,read_time,genres,genres_9,genres_18,followers,popularity,genres_1
0,fiFDr5jQg0aUSfSHw59g1rQFVSs1,2021-12-29 14:41:10.781769,2021-12-29 14:41:45.770151,2022-01-09 16:38:48.566406,"{'social media pop': 2, 'rebel blues': 4, 'cou...","{'Pop': 21, 'Rhythm music': 4, 'Alternative': ...","{'Rock': 16, 'Dance Music, Techno and House': ...",2.693735e+06,65.066667,Pop
1,Q3t2J73kpIcMqGBmB1ZEqLANtDz1,2021-12-06 18:23:20.788728,2022-01-08 11:26:15.073020,2022-01-09 16:38:48.566406,"{'miami hip hop': 3, 'nz pop': 1, 'piano rock'...","{'Pop': 30, 'Rhythm music': 1, 'Alternative': ...","{'Pop': 30, 'Utility': 1, 'Rock': 8, 'Not_assi...",1.167529e+07,77.608696,Pop
2,VQEI5Ea2AsP2tjLTlfBcmL0wn7t2,2021-12-06 15:01:05.156199,2022-01-09 15:39:58.557586,2022-01-09 16:38:48.566406,"{'social media pop': 4, 'candy pop': 1, 'nouve...","{'Pop': 25, 'Rhythm music': 1, 'Alternative': ...","{'Pop': 25, 'Heavy Metal': 10, 'Utility': 2, '...",7.156937e+06,70.812500,Rock and Heavy Metal
3,uagVX8DeaePP88Qz5Vj2EtXstK03,2021-12-11 12:21:02.186210,2021-12-11 12:21:02.186210,2022-01-09 16:38:48.566406,"{'finnish heavy metal': 1, 'rock-and-roll': 1,...","{'Pop': 10, 'Rhythm music': 3, 'Alternative': ...","{'Pop': 9, 'Heavy Metal': 21, 'Industrial and ...",4.858708e+06,69.702128,Rock and Heavy Metal
4,UrZpLI2DysWJIVCmLNwTW5JJJO62,2022-01-09 15:36:20.930481,2022-01-09 15:36:20.930481,2022-01-09 16:38:48.566406,"{'filter house': 2, 'new rave': 1, 'japanese c...","{'Pop': 11, 'Rhythm music': 4, 'Alternative': ...","{'Rock': 14, 'Dance Music, Techno and House': ...",2.468938e+06,63.487805,Not_assignable_no_concert_music
5,f6TABm8ygYT5P1NbtF7DQEk6bv82,2021-12-06 18:36:31.549954,2021-12-25 13:56:33.876422,2022-01-09 16:38:48.566406,"{'italian pop': 1, 'melodic hard rock': 1, 'in...","{'Pop': 1, 'Rock and Heavy Metal': 2}","{'Pop': 1, 'Rock': 2}",2.492494e+06,53.500000,Rock and Heavy Metal


In [7]:
spotify_renamed = spotify_extracted

spotify_renamed = spotify_renamed.withColumnRenamed("genres_9","spotify_genres_9") \
    .withColumnRenamed("genres_18","spotify_genres_18") \
    .withColumnRenamed("uid","spotify_uid") \
    .withColumnRenamed("create_time","spotify_create_time") \
    .withColumnRenamed("update_time","spotify_update_time") \
    .withColumnRenamed("read_time","spotify_read_time") \
    .withColumnRenamed("genres","spotify_genres") \
    .withColumnRenamed("followers","spotify_followers") \
    .withColumnRenamed("popularity","spotify_popularity") \
    .withColumnRenamed("genres_1","spotify_genres_1")
spotify_renamed.printSchema()

root
 |-- spotify_uid: string (nullable = true)
 |-- spotify_create_time: timestamp (nullable = true)
 |-- spotify_update_time: timestamp (nullable = true)
 |-- spotify_read_time: timestamp (nullable = true)
 |-- spotify_genres: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)
 |-- spotify_genres_9: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)
 |-- spotify_genres_18: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)
 |-- spotify_followers: double (nullable = true)
 |-- spotify_popularity: double (nullable = true)
 |-- spotify_genres_1: string (nullable = true)



In [8]:
spotify_renamed.write.format("delta").mode("overwrite").option("mergeSchema", "true").save("/home/mlops/project/DeltaLake/platinum_data/spotify_genre_agg_table")

In [9]:

spotify = spark.read.format('delta').load('/home/mlops/project/DeltaLake/platinum_data/spotify_genre_agg_table')
spotify.to_pandas_on_spark().head()


,spotify_uid,spotify_create_time,spotify_update_time,spotify_read_time,spotify_genres,spotify_genres_9,spotify_genres_18,spotify_followers,spotify_popularity,spotify_genres_1
0,fiFDr5jQg0aUSfSHw59g1rQFVSs1,2021-12-29 14:41:10.781769,2021-12-29 14:41:45.770151,2022-01-09 16:38:48.566406,"{'social media pop': 2, 'rebel blues': 4, 'cou...","{'Pop': 21, 'Rhythm music': 4, 'Alternative': ...","{'Rock': 16, 'Dance Music, Techno and House': ...",2.693735e+06,65.066667,Pop
1,Q3t2J73kpIcMqGBmB1ZEqLANtDz1,2021-12-06 18:23:20.788728,2022-01-08 11:26:15.073020,2022-01-09 16:38:48.566406,"{'miami hip hop': 3, 'nz pop': 1, 'piano rock'...","{'Pop': 30, 'Rhythm music': 1, 'Alternative': ...","{'Pop': 30, 'Utility': 1, 'Rock': 8, 'Not_assi...",1.167529e+07,77.608696,Pop
2,VQEI5Ea2AsP2tjLTlfBcmL0wn7t2,2021-12-06 15:01:05.156199,2022-01-09 15:39:58.557586,2022-01-09 16:38:48.566406,"{'social media pop': 4, 'candy pop': 1, 'nouve...","{'Pop': 25, 'Rhythm music': 1, 'Alternative': ...","{'Pop': 25, 'Heavy Metal': 10, 'Utility': 2, '...",7.156937e+06,70.812500,Rock and Heavy Metal
3,uagVX8DeaePP88Qz5Vj2EtXstK03,2021-12-11 12:21:02.186210,2021-12-11 12:21:02.186210,2022-01-09 16:38:48.566406,"{'finnish heavy metal': 1, 'rock-and-roll': 1,...","{'Pop': 10, 'Rhythm music': 3, 'Alternative': ...","{'Pop': 9, 'Heavy Metal': 21, 'Industrial and ...",4.858708e+06,69.702128,Rock and Heavy Metal
4,UrZpLI2DysWJIVCmLNwTW5JJJO62,2022-01-09 15:36:20.930481,2022-01-09 15:36:20.930481,2022-01-09 16:38:48.566406,"{'filter house': 2, 'new rave': 1, 'japanese c...","{'Pop': 11, 'Rhythm music': 4, 'Alternative': ...","{'Rock': 14, 'Dance Music, Techno and House': ...",2.468938e+06,63.487805,Not_assignable_no_concert_music
